In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install kaleido

     |████████████████████████████████| 79.9 MB 76.5 MB/s            


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import datasets, layers, models
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_curve, auc
import plotly.graph_objects as go
import glob
import os

In [4]:
def create_dir (direct):
  if not os.path.exists(direct):
    os.mkdir(direct)

In [5]:
ROOT_DIR = "/kaggle/input/ferset" #MT Directory
DATASET_02 = ROOT_DIR + "/02_FER"
DATASET_02_TRAIN = DATASET_02 + "/train"
DATASET_02_TEST = DATASET_02+ "/test"

In [6]:
import PIL

image = PIL.Image.open(DATASET_02_TRAIN + "/surprise/Training_99984132.jpg")
#image to open

width, height = image.size

print(image.mode)
print(width, height)

L
48 48


In [7]:
datagen = ImageDataGenerator()
image_size = (48, 48)
colormode = 'rgb'
test_set = datagen.flow_from_directory(DATASET_02_TEST, target_size=image_size, color_mode =colormode, batch_size=32, shuffle=False)
train_set = datagen.flow_from_directory(DATASET_02_TRAIN, target_size=image_size, color_mode =colormode, batch_size=32)

Found 7178 images belonging to 7 classes.
Found 28709 images belonging to 7 classes.


In [8]:
def create_model_V1():
    model = models.Sequential()
    model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    #model.add(layers.Dropout(0.4))
    #model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    #model.add(layers.MaxPooling2D((2, 2)))
    #model.add(layers.Dropout(0.4))
    #model.add(layers.Conv2D(16, (3, 3), activation='relu'))
    #model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(7, activation='softmax'))
    
    return model


model_1 = create_model_V1()
model_1.summary()

2022-04-09 20:07:26.279332: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-09 20:07:26.383764: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-09 20:07:26.384632: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 46, 46, 64)        1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 21, 21, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 12800)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               3277056   
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 1

2022-04-09 20:07:26.386328: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-09 20:07:26.387466: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-09 20:07:26.388169: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-09 20:07:26.388818: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

In [9]:
def get_data_df(dir):
  # modified code from: https://www.kaggle.com/namgalielei/simple-load-images-and-count-number-of-each-class

  train_df = pd.DataFrame()

  trainset = glob.glob(dir)
  #print(trainset)
  train_df['filename'] = [img.split("/")[-2]+"/"+img.split("/")[-1] for img in trainset]
  train_df['label'] = [img.split("/")[-2] for img in trainset]

  return train_df

In [10]:
test_train = get_data_df(DATASET_02_TRAIN+"/*/*.jpg")
test_train

,filename,label
0,surprise/Training_66056468.jpg,surprise
1,surprise/Training_26522394.jpg,surprise
2,surprise/Training_63239185.jpg,surprise
3,surprise/Training_67725241.jpg,surprise
4,surprise/Training_71308969.jpg,surprise
...,...,...
28704,happy/Training_18866880.jpg,happy
28705,happy/Training_16513509.jpg,happy
28706,happy/Training_10834907.jpg,happy
28707,happy/Training_30459679.jpg,happy


In [11]:
total = 0
value_list = []
for item in os.listdir(DATASET_02_TRAIN):
    y = len(os.listdir(DATASET_02_TRAIN+'/'+item))+len(os.listdir(DATASET_02_TEST+'/'+item))
    value_list.append(y)
    total += y
    print(item)
print(total)
print(value_list)
weights = {}
for n in range(0,len(value_list)):
    w = total/(len(value_list)*value_list[n])
    weights[n] = w
    
print(weights)

surprise
fear
angry
neutral
sad
disgust
happy
35887
[4002, 5121, 4953, 6198, 6077, 547, 8989]
{0: 1.2810380524023703, 1: 1.001115853488437, 2: 1.035072539009547, 3: 0.8271562255105334, 4: 0.8436258492207152, 5: 9.372420997649517, 6: 0.5703319930708962}


In [12]:
create_dir('/kaggle/working/Saved_CNN_Models')


In [13]:
from sklearn.model_selection import KFold, StratifiedKFold

train_data = get_data_df(DATASET_02_TRAIN+"/*/*.jpg")
Y = train_data[['label']]

#Y = final_train[['label']]
#n = len(final_train)   

skf = StratifiedKFold(n_splits = 5, random_state = 7, shuffle = True) 

#Un needed preprocessing aside from rescale
#idg = ImageDataGenerator(rescale=1./255)
create_dir('/kaggle/working/Saved_CNN_Models')
#model name
def get_model_name(k):
    return 'model_'+str(k)+'.h5'
def model_test(fold_count, epoch, model_method, image_size, mode):
    skf = StratifiedKFold(n_splits = fold_count, random_state = 7, shuffle = True)
    train_data = get_data_df(DATASET_02_TRAIN+"/*/*.jpg")
      #Incorperate preprocessing in the method 
      #####################################################################
      #aug_train = get_data_df(DATASET_01_ROOT+"/Aug_train/*/*.jpg")
      #aug_train['filename'] = aug_train['filename'].str[7:]
      #final_train = test_train.append(aug_train, sort=False, ignore_index=True)


      #######################################################################
      #idg = ImageDataGenerator(rescale=1./255)
    idg = ImageDataGenerator()
    Y = train_data[['label']]
    n = len(train_data)    
    VALIDATION_ACCURACY = []
    VALIDATION_LOSS = []

      #create a directory
    save_dir = '/kaggle/working/Saved_CNN_Models/'+model_method.__name__+'_'+str(epoch)+'epochs/'
    create_dir(save_dir)
    
    img_dir = DATASET_02_TRAIN

    fold_var = 1

      #n is number of samples, doesn't care about x values for split, just y
    for train_index, val_index in skf.split(np.zeros(n),Y):

        #Index value for fold
        training_data = train_data.iloc[train_index]
        validation_data = train_data.iloc[val_index]
        #generator for train
        print("hi")
        print(validation_data.head())
        #Modify the training size, have an input adjustability 
        train_data_generator = idg.flow_from_dataframe(training_data, directory = img_dir,
                                                       target_size=image_size, batch_size=32, color_mode =mode,
                                                       x_col = "filename", y_col = "label",shuffle = True)
        #print("hi")
        #generator for validation
        valid_data_generator  = idg.flow_from_dataframe(validation_data, directory = img_dir,
                                                        target_size=image_size, color_mode =mode,batch_size=32,
                                                        x_col = "filename", y_col = "label",shuffle = True)
        #print("hi")
        #create a method for this
        # CREATE NEW MODEL
        model = model_method()
        # COMPILE NEW MODEL
        model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                optimizer= tf.keras.optimizers.Adam(learning_rate=0.0001),
                metrics=['accuracy'])

        # CREATE CALLBACKS
        #Unsure about checpoint
        checkpoint = tf.keras.callbacks.ModelCheckpoint(save_dir+model_method.__name__+"_"+get_model_name(fold_var), 
                    monitor='val_accuracy', verbose=1, 
                    save_best_only=True, mode='max')

        callbacks_list = [checkpoint]
        # There can be other callbacks, but just showing one because it involves the model name
        # This saves the best model
        # FIT THE MODEL
        #Include the weights for classes
        history = model.fit(train_data_generator,
                epochs=epoch,
                callbacks=callbacks_list,
                validation_data=valid_data_generator
                ,class_weight = weights
                ,verbose = 2)

        #PLOT HISTORY
        #Plots for whichever
        #		:
        #plt.plot(history.history['binary_accuracy'], label='accuracy')
        #plt.plot(history.history['val_binary_accuracy'], label = 'val_accuracy')
        #plt.xlabel('Epoch')
        #plt.ylabel('Accuracy')
        #plt.ylim([0.3, 1])
        #plt.legend(loc='lower right')
        #plt.show()
        #frame = {'Accuracy': history.history['binary_accuracy'], 'Val_Accuracy': history.history['val_binary_accuracy']}
        #fig = px.line(frame)
        #fig.show()
        fig = go.Figure()
        fig.add_trace(go.Scatter(y= history.history['accuracy'],
                          mode='lines+markers',
                          name='Training Accuracy'))
        fig.add_trace(go.Scatter(y= history.history['val_accuracy'],
                          mode='lines+markers',
                          name='Validation Accuracy'))
        fig.update_yaxes(range=[0, 1])
        fig.update_layout(
            title="Fold "+ str(fold_var)+ "'s performance",
            title_x=0.5,
            xaxis_title="Epochs",
            yaxis_title="Accuracy",
            autosize=False,
            width=800,
            height=400,
            margin=dict(l=20, r=20, t=40, b=20),)
        fig.show()
        graph_dir = '/kaggle/working/img'
        create_dir(graph_dir)
        fig.write_image(graph_dir+'/fold'+str(fold_var)+'.png') 

        #		:
        # LOAD BEST MODEL to evaluate the performance of the model
        model.load_weights(save_dir+model_method.__name__+"_" + "model_"+str(fold_var)+".h5")

        results = model.evaluate(test_set)
        results = dict(zip(model.metrics_names,results))

        VALIDATION_ACCURACY.append(results['accuracy'])
        VALIDATION_LOSS.append(results['loss'])

        tf.keras.backend.clear_session()

        fold_var += 1

In [14]:
#model_test(5, 1, create_model_V1, (48,48), 'rgb')

In [15]:
def transfer_learning_model_resnet_50():

  IMG_SHAPE = (48,48,3,)
 
  inputs = tf.keras.Input(shape=IMG_SHAPE)
  #inputs = tf.image.grayscale_to_rgb(y)
 
  x = tf.keras.applications.resnet50.preprocess_input(inputs)
  base_model = tf.keras.applications.resnet50.ResNet50(input_shape=(48,48,3,),
                                               include_top=False,
                                               weights='imagenet')
  #print(len(base_model.layers))
  base_model.trainable = True

  fine_tune_at = 400
  for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

  x = base_model(x)
  global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
  x = global_average_layer(x)
  x = tf.keras.layers.Dropout(0.2)(x)
  prediction_layer = tf.keras.layers.Dense(7, activation = "softmax")
  outputs = prediction_layer(x)
  model = tf.keras.Model(inputs, outputs)
  #print(len(model.trainable_variables))
  return model

transfer_model_resnet_50 = transfer_learning_model_resnet_50()

94781440/94765736 [==============================] - 0s 0us/step


In [16]:
transfer_model_resnet_50.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
tf.__operators__.getitem (Sl (None, 48, 48, 3)         0         
_________________________________________________________________
tf.nn.bias_add (TFOpLambda)  (None, 48, 48, 3)         0         
_________________________________________________________________
resnet50 (Functional)        (None, 2, 2, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 14343 

In [17]:
model_test(5, 20, transfer_learning_model_resnet_50, (48,48,), 'rgb')

hi
                          filename     label
10  surprise/Training_52655852.jpg  surprise
16  surprise/Training_44186778.jpg  surprise
21  surprise/Training_87248788.jpg  surprise
31  surprise/Training_32069005.jpg  surprise
45   surprise/Training_4473463.jpg  surprise
Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.


2022-04-09 20:07:47.442832: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20


2022-04-09 20:07:51.042097: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


718/718 - 116s - loss: 4.6502 - accuracy: 0.1951 - val_loss: 2.9749 - val_accuracy: 0.1984

Epoch 00001: val_accuracy improved from -inf to 0.19836, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_20epochs/transfer_learning_model_resnet_50_model_1.h5


...

Epoch 2/20
718/718 - 25s - loss: 3.4340 - accuracy: 0.2333 - val_loss: 2.7215 - val_accuracy: 0.2074

Epoch 00002: val_accuracy did not improve from 0.21944
Epoch 3/20
718/718 - 25s - loss: 3.1092 - accuracy: 0.2471 - val_loss: 2.4754 - val_accuracy: 0.2215

Epoch 00003: val_accuracy improved from 0.21944 to 0.22153, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_20epochs/transfer_learning_model_resnet_50_model_4.h5
Epoch 4/20
718/718 - 25s - loss: 2.8871 - accuracy: 0.2547 - val_loss: 2.2895 - val_accuracy: 0.2478

Epoch 00004: val_accuracy improved from 0.22153 to 0.24782, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_20epochs/transfer_learning_model_resnet_50_model_4.h5
Epoch 5/20
718/718 - 25s - loss: 2.7661 - accuracy: 0.2592 - val_loss: 2.3064 - val_accuracy: 0.2339

Epoch 00005: val_accuracy did not improve from 0.24782
Epoch 6/20
718/718 - 25s - loss: 2.6712 - accuracy: 0.2637 - val_loss: 2.1679 - val_accu

225/225 [==============================] - 7s 30ms/step - loss: 1.8926 - accuracy: 0.3091
hi
                          filename     label
1   surprise/Training_26522394.jpg  surprise
5   surprise/Training_49242724.jpg  surprise
13   surprise/Training_7022565.jpg  surprise
15  surprise/Training_37026288.jpg  surprise
17  surprise/Training_97667504.jpg  surprise
Found 22968 validated image filenames belonging to 7 classes.
Found 5741 validated image filenames belonging to 7 classes.
Epoch 1/20
718/718 - 29s - loss: 4.6149 - accuracy: 0.1954 - val_loss: 2.7739 - val_accuracy: 0.2132

Epoch 00001: val_accuracy improved from -inf to 0.21320, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_20epochs/transfer_learning_model_resnet_50_model_5.h5


/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning:

Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.



Epoch 2/20
718/718 - 25s - loss: 3.4560 - accuracy: 0.2312 - val_loss: 2.4276 - val_accuracy: 0.2379

Epoch 00002: val_accuracy improved from 0.21320 to 0.23794, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_20epochs/transfer_learning_model_resnet_50_model_5.h5
Epoch 3/20
718/718 - 25s - loss: 3.1551 - accuracy: 0.2444 - val_loss: 2.6300 - val_accuracy: 0.2165

Epoch 00003: val_accuracy did not improve from 0.23794
Epoch 4/20
718/718 - 25s - loss: 2.9420 - accuracy: 0.2527 - val_loss: 2.4424 - val_accuracy: 0.2327

Epoch 00004: val_accuracy did not improve from 0.23794
Epoch 5/20
718/718 - 25s - loss: 2.7775 - accuracy: 0.2631 - val_loss: 2.2618 - val_accuracy: 0.2353

Epoch 00005: val_accuracy did not improve from 0.23794
Epoch 6/20
718/718 - 25s - loss: 2.6581 - accuracy: 0.2629 - val_loss: 2.3802 - val_accuracy: 0.2313

Epoch 00006: val_accuracy did not improve from 0.23794
Epoch 7/20
718/718 - 25s - loss: 2.5782 - accuracy: 0.2690 - val_loss: 2.

225/225 [==============================] - 7s 30ms/step - loss: 1.9437 - accuracy: 0.2933
